In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [1]:
from fosforio import snowflake
from fosforio import get_dataframe
from datetime import datetime, timedelta

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns

Matplotlib created a temporary cache directory at /tmp/matplotlib-vhjm44ak because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS")
data

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7fb2acc528e0>
Please close the connection after use!
Reading dataframe from snowflake native connector


,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,City Hotel,0,28,2022,10,41,6,2023-08-06 00:00:00,Check.Out,07-10-2022,1,1.000,0,1,2,0,0,2,1,1,SC,BRA,Online TA,TA/TO,0,0,A,G,No Deposit,0,Transient,129.00
1,Resort Hotel,0,32,2022,10,41,3,2023-08-03 00:00:00,Check.Out,07-10-2022,4,4.000,1,3,2,2,0,4,2,8,BB,IRL,Online TA,TA/TO,0,0,G,G,No Deposit,0,Transient,170.50
2,Resort Hotel,1,221,2023,4,18,30,2024-02-28 00:00:00,Canceled,07-10-2022,7,0.205,2,5,2,0,0,2,1,7,HB,PRT,Groups,TA/TO,0,0,D,D,No Deposit,0,Transient.Party,82.00
3,City Hotel,1,115,2022,10,42,13,2023-08-13 00:00:00,Canceled,07-10-2022,1,0.600,0,1,2,0,0,2,1,1,SC,FRA,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,108.00
4,Resort Hotel,0,118,2022,10,41,4,2023-08-04 00:00:00,Check.Out,07-10-2022,3,3.000,0,3,2,0,0,2,1,3,BB,IRL,Offline TA/TO,TA/TO,0,0,A,G,No Deposit,0,Transient,46.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117004,City Hotel,1,111,2022,10,42,10,2023-08-10 00:00:00,Canceled,07-10-2022,1,0.300,1,0,1,0,0,1,1,1,BB,PRT,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient.Party,120.00
117005,City Hotel,1,109,2022,10,42,11,2023-08-11 00:00:00,Canceled,07-10-2022,1,0.400,0,1,2,0,0,2,1,1,SC,FRA,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,108.00
117006,City Hotel,0,222,2022,10,41,2,2023-08-02 00:00:00,Check.Out,07-10-2022,5,5.000,2,3,2,0,0,2,1,5,BB,DEU,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,102.85
117007,City Hotel,1,30,2022,10,43,20,2023-08-20 00:00:00,Canceled,07-10-2022,3,0.130,0,3,2,1,0,3,1,3,BB,GBR,Online TA,TA/TO,0,0,A,A,No Deposit,0,Transient,159.00


In [7]:
data = pd.read_csv('./bookings.csv')
data

,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,City Hotel,1,272,2021,7,29,16,2021-07-16,Canceled,17-10-2020,2,0.272,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
1,City Hotel,1,307,2021,8,34,20,2021-08-20,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
2,City Hotel,1,258,2021,7,27,2,2021-07-02,Canceled,17-10-2020,2,0.258,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
3,City Hotel,1,307,2021,8,34,20,2021-08-20,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
4,City Hotel,1,279,2021,7,30,23,2021-07-23,Canceled,17-10-2020,2,0.279,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117100,Resort Hotel,0,269,2023,8,34,24,2023-08-24,Check.Out,10-09-2023,17,17.000,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
117101,Resort Hotel,0,212,2023,8,35,31,2023-08-31,Check.Out,10-09-2023,10,10.000,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
117102,Resort Hotel,0,204,2023,8,35,29,2023-08-29,Check.Out,12-09-2023,14,14.000,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
117103,Resort Hotel,0,161,2023,8,35,31,2023-08-31,Check.Out,14-09-2023,14,14.000,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06


In [8]:
data.columns

Index(['HOTEL', 'IS_CANCELED', 'LEAD_TIME', 'ARRIVAL_DATE_YEAR', 'MONTH',
       'ARRIVAL_DATE_WEEK_NUMBER', 'ARRIVAL_DATE_DAY_OF_MONTH',
       'EXPECTED_ARRIVAL_DATE', 'RESERVATION_STATUS',
       'RESERVATION_STATUS_DATE', 'TOTAL_STAY_NIGHTS', 'TALLY_DAYS',
       'STAYS_IN_WEEKEND_NIGHTS', 'STAYS_IN_WEEK_NIGHTS', 'ADULTS', 'CHILDREN',
       'BABIES', 'TOTAL_GUESTS', 'AVG_ROOMS_PER_NIGHT', 'TOTAL_ROOM_NIGHTS',
       'MEAL', 'COUNTRY', 'MARKET_SEGMENT', 'DISTRIBUTION_CHANNEL',
       'PREVIOUS_CANCELLATIONS', 'PREVIOUS_BOOKINGS_NOT_CANCELED',
       'RESERVED_ROOM_TYPE', 'ASSIGNED_ROOM_TYPE', 'DEPOSIT_TYPE',
       'DAYS_IN_WAITING_LIST', 'CUSTOMER_TYPE', 'ADR'],
      dtype='object')

In [9]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE']).min()

Timestamp('2021-07-01 00:00:00')

In [10]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE']).max()

Timestamp('2023-08-31 00:00:00')

In [11]:
data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')


In [12]:
data.sort_values(by='EXPECTED_ARRIVAL_DATE')['EXPECTED_ARRIVAL_DATE']

2819    2021-07-01
2953    2021-07-01
1427    2021-07-01
2950    2021-07-01
2949    2021-07-01
           ...    
34089          NaT
34149          NaT
34275          NaT
34319          NaT
34859          NaT
Name: EXPECTED_ARRIVAL_DATE, Length: 117105, dtype: datetime64[ns]

In [13]:
original_last_date = datetime(2023, 8, 31)
target_date = datetime(2024, 6, 30)

In [14]:
original_min_date = data['EXPECTED_ARRIVAL_DATE'].min()
original_min_date

Timestamp('2021-07-01 00:00:00')

In [15]:
days_to_add = (target_date - original_last_date).days
days_to_add

304

In [17]:
data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')

<ipython-input-17-59d974ffbd72>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['RESERVATION_STATUS_DATE'] = pd.to_datetime(data['RESERVATION_STATUS_DATE'], errors='coerce')


In [18]:
data['EXPECTED_ARRIVAL_DATE'] = data['EXPECTED_ARRIVAL_DATE'] + timedelta(days=days_to_add)
data['RESERVATION_STATUS_DATE'] = data['RESERVATION_STATUS_DATE'] + timedelta(days=days_to_add)

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].min()

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].max()

In [ ]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

In [ ]:
data.info()

In [ ]:
data["EXPECTED_ARRIVAL_DATE"] = pd.to_datetime(data["EXPECTED_ARRIVAL_DATE"]).dt.strftime("%Y-%m-%d %H:%M:%S")


In [ ]:
data.info()

In [ ]:
df_model=session.createDataFrame(
        data_backup.values.tolist(),
        schema=data_backup.columns.tolist())
df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.BOOKINGS")

In [ ]:
data['EXPECTED_ARRIVAL_DATE'].isna().sum()

In [ ]:
data_backup = data.dropna(axis=0)

In [ ]:
data_backup['EXPECTED_ARRIVAL_DATE'].min()

In [ ]:
data_backup['EXPECTED_ARRIVAL_DATE'].max()

In [ ]:
117105 - 117009